# Learning goals
After this week's lesson you should be able to:
- Perform an overlay analysis
- Reclassify data
- Perform a spatial join (Refresher)

This week's lessons are adapted from:
- [Automating GIS Processes Lesson 3](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-3/overview.html)
- [Automating GIS Processes Lesson 4](https://autogis-site.readthedocs.io/en/latest/lessons/lesson-4/overview.html)

In [ ]:
# We are going to start importing the libraries we need
# all in one cell. 
# It is a good practice to keep all the imports in one cell so that
# we can easily see what libraries we are using in the notebook.

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

import os
os.getcwd()

# 1. Overlay Analysis
Overlay analyses are GIS operations in which two or more vector layers are combined to produce new geometries. Typical overlay operations include union, intersection, and difference - named after the result of the combination of two layers.

</figure>
<img src="https://autogis-site.readthedocs.io/en/latest/_images/overlay-operations_700x200px.svg" alt="drawing" width="700" style="display: block; margin: 0 auto"/>
</figure>

## 1.1 Loading data
Let's say we wanted to study which [neighborhoods](https://data.cityofnewyork.us/City-Government/NTA-map/d3qk-pfyz) in NYC are more vulnerable to flooding. Here, we will measure vulnerability as the areal coverage of the neighborhood by the [NYC Stormwater Flood Map](https://data.cityofnewyork.us/City-Government/NYC-Stormwater-Flood-Map-Moderate-Flood-with-2050-/5rzh-cyqd). 

Let's download two datasets and put it into our current folder: 
- The [Neighborhood Tabulation Areas](https://data.cityofnewyork.us/City-Government/NTA-map/d3qk-pfyz) (make sure to download the shapefile)
- [NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise](https://data.cityofnewyork.us/City-Government/NYC-Stormwater-Flood-Map-Moderate-Flood-with-2050-/5rzh-cyqd) There is a zip file in on this page and you will have to unzip to read a `.gdb` file

In [ ]:
nta = gpd.read_file('NTA map.zip')

In [ ]:
nta.head()

In [ ]:
nta.plot()

In [ ]:
## I'm sorry this file is a pain 
## It is a .gdb file, which is a geodatabase file in ArcGIS
## It only has one layer, so we can just read it in with layer=0
## We can also use the driver='FileGDB' to tell geopandas that it is a geodatabase file
## Though Geopandas will infer the file format from the file extension, so we don't need to specify the driver 



# flood_2050 = gpd.read_file("NYC_Stormwater_Flood_Map_-_Moderate_Flood_with_2050_Sea_Level_Rise/NYC Stormwater Flood Map - Moderate Flood with 2050 Sea Level Rise.gdb",
#                         driver='FileGDB', 
#                         layer=0)

flood_2050 = gpd.read_file('flood_2050.geojson',driver='GeoJSON')

# And this file name is terrible. 
# Ok enough griping. 

In [ ]:
flood_2050

In [ ]:
flood_2050.to_file('flood_2050.geojson',driver='GeoJSON')

In [ ]:
flood_2050 = flood_2050.rename(columns={'Flooding_C':'Flooding_Category'})

In [ ]:
flood_2050.head()

This is unusual. There are only three rows in the dataset! Judging by the `Flooding_Category` column an the fact that each entry in the `geometry` column is a **MULTIPOLYGON** (we're going to ignore the Z part. This is actually a 3D polygon, but we won't work with the third dimension in this example.)

Looking, in the data dictionary for the flood data, which should be the `.xlsx` file in the unzipped folder we can see that these are the following categories definitions: 
- 1 - Nuisance Flooding (greater or equal to 4 in. and less than 1 ft.)
- 2 - Deep and Contiguous Flooding (1 ft. and greater)
- 3 - Future High Tides 2050

I'm going to make a quick categorical plot to get a sense of what the data may look like. 

In [ ]:
flood_2050.plot('Flooding_Category',
                figsize=(10,10),
                legend=True,
                categorical=True)

For now, let's just use **Category 1 "Nuisance Flooding"** to simplify this calculation. I'm going to create a new gdf called `flood_2050_cat1` 

In [ ]:
flood_2050['Flooding_Category'] == 1

In [ ]:
flood_2050_cat1 = flood_2050[flood_2050['Flooding_Category'] == 1]

## 1.2 `.overlay()`
We'll want to estimate the **percentage of the neighborhood at risk of category 1 flooding**. The first step is to do the following: 
1. Check the CRS between our two layers to make sure they are the same. 
2. Find the **Intersection** of geometries, using the `.overlay()`, function between the neighborhoods and the flood zones: That is we'll want to find the shapes that overlap between the neighborhood and each flood zone. 

In [ ]:
nta.crs

In [ ]:
flood_2050_cat1.crs

Looks like they are different: 
- `nta` is **EPSG:4326**
- `flood_2050` is **EPSG:2263**

Notice that EPSG:2263 is in **feet**. Again, whenever we want to estimate areas, volumes, or lengths, we'll want to do this in a CRS in a unit that we can understand. (Areas in degrees doesn't make too much sense.) 

For now, let's convert the `nta` to **EPSG:2263** to have our measurements in **feet**. I'm going to create a new gdf `nta_2263`. 

In [ ]:
nta_2263 = nta.to_crs(flood_2050_cat1.crs)

Finally, I want to create this overlay and assign it to a new gdf called `nta_2263_overlay_flood`. Note: This may take little bit of time as the `.overlay()` function is calculating all possible matches. Between geometries in `nta_2263` and `flood_2050_cat1`. (It took about 45 seconds on my computer.)

In [ ]:
nta_2263_overlay_flood = gpd.overlay(nta_2263,flood_2050_cat1,how='intersection')

In [ ]:
nta_2263_overlay_flood.head()

## Q.1 Groupby-and-summarize (5 pts)
How many rows for each neighborhood are there? `nta_2263_overlay_flood`? 

In [ ]:
## insert your code here

Let's take a look at what resulted. We can see here that we have columns from both the neighborhoods and flood gdfs. 

In [ ]:
nta_2263_overlay_flood.head()

In [ ]:
## I'm just going to select one neighborhood and see what this overlay looks like
fig1, ax1 = plt.subplots(1,1,figsize=(10,10))

## Plotting the neighborhood BK43
nta_2263[nta_2263['ntacode']=='BK75'].plot(ax=ax1)

## Plotting the flood zone intersecting with the neighborhood
nta_2263_overlay_flood[nta_2263_overlay_flood['ntacode']=='BK75'].plot(ax=ax1,color='red')

## 1.3 Find area percentage
For all neighborhoods, I'll want to find: 
$$
\% floodzone_{n} =\frac{A_{fn}}{A_{n}}
$$

where $n$ is a neighborhood, $A_{fn}$ is the area of the flood zone that intersects with that neighborhood $n$ and $A_n$ is the area of the neighorhood.

(Yes we can write mathematical notation in Markdown using LaTex! No, I won't make you do it if you aren't already familiar with LaTex.)

To do this, we'll need: 
- $A_{fn}$ 
- $A_n$

In [ ]:
nta_2263_overlay_flood.area

In [ ]:
nta_2263_overlay_flood['area_flood'] = nta_2263_overlay_flood.area
nta_2263['area_neighb'] = nta_2263.area

In [ ]:
nta_2263_overlay_flood.head()

## 1.4 Merge
I'm now going merge my `nta_2263_overlay_flood` gdf with the `nta_2263` because I want to divide `area_flood` and `area_neighb`. 
 

In [ ]:
## Again I'm just going to select the two columns I need, the neighborhood code (to perform the merge) 
## and the area of the flood zone (to calculate the percentage of the neighborhood that is flooded)
## I'm going to use a left join, so that I keep all the neighborhoods in the original dataset, 
## even if they don't intersect with the flood zone dataset

nta_2263_merged = nta_2263.merge(nta_2263_overlay_flood[['ntacode','area_flood']],
                                 left_on='ntacode',
                                 right_on='ntacode',how='left')

Super! 

In [ ]:
nta_2263_merged.head()

## 1.5 Calculate percentage
Lastly, we'll just have to calculate the percentage of the flood zone in each neighborhood. 


In [ ]:
nta_2263_merged['perc_flood'] = nta_2263_merged['area_flood'] / nta_2263_merged['area_neighb']

In [ ]:
nta_2263_merged.plot(column='perc_flood', 
                        legend=True,
                        figsize=(10,10))

## Q.2 - Topical Knowledge (5 pts)
Here we used flood zone coverage in a neighborhood as proxy for flood. Why might this be an inexact estimate of flood risk? Propose and describe at least three factors we may not have considered in this analysis. 

## 1.5 Re-classifying data
Now, instead of using the `perc_flood` column, I want to translate the values in this column into categories that might be more meaningful to a general audience. 

- We might consider binning our values into "Low", "Average", and "High" risk categories. How to determine these? The best approach is have some topical knowledge. For instance, we might read from research reports, look at previous flooding records of these neigbhohoods, etc, to acquire an understanding of how to bin values. 
- A less informed strategy could be to categorize by the distribution of our data. 

We'll take a the less informed strategy here for the sake of time: 

In [ ]:
## Remember that the describe function gives us some basic statistics about the data
nta_2263_merged['perc_flood'].describe()

From the above, we'll use the following criteria: 
- Less than 25th percentile = Low
- 25-75th percentile = Average
- 75th percentile or higher = High

To express this in code. We will: 
- Create a new empty column 
- Filter our gdf based on each criteria
- Assign a different category to each criteria. 

### 1.5.1
First, let's create an empty string column.

In [ ]:
# Remember that "" is an empty string
nta_2263_merged['risk_categories'] = ""

See that we have a column called `risk_categories` that contains empty strings (note: not the same as `NaN`)

In [ ]:
nta_2263_merged.head()

### 1.5.2 
Now we'll filter for each criteria

In [ ]:
## I'm going to use .loc instead of the square bracket method to select the rows and columns I want to edit
## Remember that .loc is used to select rows and columns by label

## So I simultaneously select my filtering criteria for rows and the column I want to edit. 
## 0.004067 is the 25th percentile of the data
nta_2263_merged.loc[nta_2263_merged['perc_flood']<0.004067,'risk_categories'] = 'low'

## A lot of things are happening here: 
## 1. We're filtering for two conditions: Both >=25% and <=75% 
## 2. Because I have multiple conditions, I need to use the & operator to combine them 
## and use the () on each condition
## 3. I'm breaking up my code into multiple lines for readability with the \ character
nta_2263_merged.loc[(nta_2263_merged['perc_flood']>=0.004067)&\
                    (nta_2263_merged['perc_flood']<=0.016422),'risk_categories'] = 'average'

## Always make sure that you're not double counting your rows
## If you use a <=X condition, you'll need to use a >X condition
nta_2263_merged.loc[nta_2263_merged['perc_flood']>0.016422,'risk_categories'] = 'high'

In [ ]:
nta_2263_merged.head()

In [ ]:
nta_2263_merged.plot(column='risk_categories', 
                        legend=True,
                        categorical=True,
                        figsize=(10,10))

## Q.3 - Missing data (2 pts)
Oops, how do we find those neighborhoods that weren't categorized? 

In [ ]:
## insert your code here

## Optional: Q.4 - Classifying Data (5 pts)
How would you recategorize those empty neighborhoods? Map the re-categorized data. 

In [ ]:
## insert your code here

## Q.5 (10pts) - Spatial join
- Download the [Points of Interest](https://data.cityofnewyork.us/City-Government/Points-Of-Interest/rxuy-2muj) dataset from the NYC OpenData portal
- Download the [NYCHA Developments](https://data.cityofnewyork.us/Housing-Development/Map-of-NYCHA-Developments/i9rv-hdr5) dataset again. 




For each NYCHA development, calculate the number of POIs within a 15 minute walkshed. Which development has the most POIs within a 15 minute walkshed? Which development has the least? 

In [ ]:
## insert your code here